In [137]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier


In [138]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

In [139]:
train_df=train_df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)
test_df=test_df.drop(['Name','Ticket','Cabin'],axis=1)

In [140]:
label_encoder = preprocessing.LabelEncoder()
train_df['Sex']= label_encoder.fit_transform(train_df['Sex'])
test_df['Sex']= label_encoder.fit_transform(test_df['Sex'])

In [141]:
train_df['Embarked']=train_df['Embarked'].fillna(train_df['Embarked'].mode()[0])

In [142]:
categorical_columns = train_df.select_dtypes(include=['object']).columns.tolist()
encoder = OneHotEncoder(sparse_output=False)

one_hot_encoded = encoder.fit_transform(train_df[categorical_columns])

one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

train_df = pd.concat([train_df, one_hot_df], axis=1)

train_df = train_df.drop(categorical_columns, axis=1)
print(f"Encoded Employee data : \n{train_df}")

Encoded Employee data : 
     Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked_C  \
0           0       3    1  22.0      1      0   7.2500         0.0   
1           1       1    0  38.0      1      0  71.2833         1.0   
2           1       3    0  26.0      0      0   7.9250         0.0   
3           1       1    0  35.0      1      0  53.1000         0.0   
4           0       3    1  35.0      0      0   8.0500         0.0   
..        ...     ...  ...   ...    ...    ...      ...         ...   
886         0       2    1  27.0      0      0  13.0000         0.0   
887         1       1    0  19.0      0      0  30.0000         0.0   
888         0       3    0   NaN      1      2  23.4500         0.0   
889         1       1    1  26.0      0      0  30.0000         1.0   
890         0       3    1  32.0      0      0   7.7500         0.0   

     Embarked_Q  Embarked_S  
0           0.0         1.0  
1           0.0         0.0  
2           0.0         1.0  
3 

In [143]:
categorical_columns = test_df.select_dtypes(include=['object']).columns.tolist()
encoder = OneHotEncoder(sparse_output=False)

one_hot_encoded = encoder.fit_transform(test_df[categorical_columns])

one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

test_df = pd.concat([test_df, one_hot_df], axis=1)

test_df = test_df.drop(categorical_columns, axis=1)
print(f"Encoded Employee data : \n{test_df}")

Encoded Employee data : 
     PassengerId  Pclass  Sex   Age  SibSp  Parch      Fare  Embarked_C  \
0            892       3    1  34.5      0      0    7.8292         0.0   
1            893       3    0  47.0      1      0    7.0000         0.0   
2            894       2    1  62.0      0      0    9.6875         0.0   
3            895       3    1  27.0      0      0    8.6625         0.0   
4            896       3    0  22.0      1      1   12.2875         0.0   
..           ...     ...  ...   ...    ...    ...       ...         ...   
413         1305       3    1   NaN      0      0    8.0500         0.0   
414         1306       1    0  39.0      0      0  108.9000         1.0   
415         1307       3    1  38.5      0      0    7.2500         0.0   
416         1308       3    1   NaN      0      0    8.0500         0.0   
417         1309       3    1   NaN      1      1   22.3583         1.0   

     Embarked_Q  Embarked_S  
0           1.0         0.0  
1           0.

In [144]:
X_train=train_df.drop(['Survived'],axis=1)
y_train=train_df[['Survived']]

In [145]:
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(test_df.drop(['PassengerId'],axis=1))

/Users/elliebarranbeck/ds-projects/Titanic/.venv/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [146]:
submission=pd.DataFrame({
    'PassengerId': test_df.PassengerId,
    'Survived': y_pred
})
submission.to_csv('submission.csv',index=False)